In [ ]:
# from google.colab import drive

# drive.mount("/content/drive")


Mounted at /content/drive
drive  sample_data


In [ ]:
!pip install torch_xla

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 MB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install lightning

In [ ]:
!pip install open_clip_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 150.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
import math
import time
import open_clip

import numpy as np
import pandas as pd

from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from typing import Dict, List, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor

from tqdm.auto import tqdm

In [ ]:
# Torch imports
import torch
from torch import nn
from torch import amp
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR, ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast
from torchmetrics.functional import accuracy
import torch.nn.functional as F

from torchvision.models import vit_b_32, ViT_B_32_Weights

import torchvision.transforms as T

from lightning.fabric import Fabric, seed_everything
from lightning.fabric.accelerators import XLAAccelerator

In [ ]:
fabric = Fabric(
    accelerator="tpu",
    devices=1,
    precision="bf16-true",
)
fabric.device

class BioCLIPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes, freeze_backbone=True):
        super().__init__()
        self.clip_model = clip_model

        # Get the embedding dimension from the visual encoder
        embed_dim = clip_model.visual.output_dim

        # Simple classifier head
        self.head = nn.Linear(embed_dim, num_classes)

        if freeze_backbone:
            for p in self.clip_model.parameters():
                p.requires_grad = False

    def forward(self, images):
        # CLIP encodes images into embeddings
        # encode_image expects images preprocessed using `preprocess_*`
        with torch.no_grad() if not any(p.requires_grad for p in self.clip_model.parameters()) else torch.enable_grad():
            image_features = self.clip_model.encode_image(images)

        logits = self.head(image_features)
        return logits

In [ ]:
num_cpu = os.cpu_count() or 4
num_workers = max(1, num_cpu // 2)
print(f"found {num_cpu} cpus, allocation {num_workers} workers")

found 44 cpus, allocation 22 workers


In [ ]:
metadata_path = (
    "/content/drive/MyDrive/herpeton/data/biotrove_train/reptilia_dataset_processed.csv"
)
BASE_PATH = "/content/drive/MyDrive/herpeton/data/biotrove_train"
IMG_DIR = os.path.join(BASE_PATH, "images_reptilia")

df = pd.read_csv(metadata_path)
print("Metadata shape:", df.shape)
df.head()

Metadata shape: (18750, 16)


,photo_id,scientificName,kingdom,phylum,class,order,family,genus,species,common_name,taxonRank,photo_url,split,image_path_fixed,width,height
0,483440,Agkistrodon laticinctus,Animalia,Chordata,Reptilia,Squamata,Viperidae,Agkistrodon,laticinctus,Broad-banded Copperhead,species,http://inaturalist-open-data.s3.amazonaws.com/...,train,/content/drive/MyDrive/herpeton/data/biotrove_...,480.0,358.0
1,345778,Sceloporus uniformis,Animalia,Chordata,Reptilia,Squamata,Phrynosomatidae,Sceloporus,uniformis,Yellow-backed Spiny Lizard,species,http://inaturalist-open-data.s3.amazonaws.com/...,train,/content/drive/MyDrive/herpeton/data/biotrove_...,500.0,353.0
2,181353,Pantherophis spiloides,Animalia,Chordata,Reptilia,Squamata,Colubridae,Pantherophis,spiloides,Gray Ratsnake,species,http://inaturalist-open-data.s3.amazonaws.com/...,train,/content/drive/MyDrive/herpeton/data/biotrove_...,373.0,500.0
3,35083,Lampropeltis triangulum,Animalia,Chordata,Reptilia,Squamata,Colubridae,Lampropeltis,triangulum,Eastern Milksnake,species,http://inaturalist-open-data.s3.amazonaws.com/...,train,/content/drive/MyDrive/herpeton/data/biotrove_...,500.0,376.0
4,261176,Imantodes cenchoa,Animalia,Chordata,Reptilia,Squamata,Colubridae,Imantodes,cenchoa,Blunthead Tree Snake,species,http://inaturalist-open-data.s3.amazonaws.com/...,train,/content/drive/MyDrive/herpeton/data/biotrove_...,500.0,337.0


In [ ]:
# Use 'species' as label
assert "scientificName" in df.columns, "Expected 'scientificName' column in metadata."
assert "image_path_fixed" in df.columns, "Expected 'image_path_fixed' column with image paths."

# Encode species as integer labels
df = df.dropna(subset=['image_path_fixed']).reset_index(drop=True)
df["label_id"], label_names = pd.factorize(df["scientificName"])

num_classes = len(label_names)
print("Number of classes:", num_classes)

# Split data using the 'split' column
train_df = df[df["split"] == "train"].copy()
val_df = df[df["split"] == "val"].copy()
test_df = df[df["split"] == "test"].copy()

print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))


Number of classes: 531
Train size: 13879
Val size: 1982
Test size: 1753


In [ ]:
with fabric.init_module():
  biomodel, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:imageomics/bioclip', device='cpu',)
  model_bio = BioCLIPClassifier(
      clip_model=biomodel,
      num_classes=num_classes,
      freeze_backbone=True,
  )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

open_clip_pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

In [ ]:
BATCH_SIZE = 64
# Image size compatible with ViT model
image_size = (224, 224)

class ReptiliaDataset(Dataset):
    def __init__(
        self,
        df,
        cache_images=True,
        cache_workers=8,
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
        isTrain=False,
        transform=None,
    ):
        self.df = df.reset_index(drop=True).copy()
        self.mean = torch.tensor(mean).view(3,1,1)
        self.std = torch.tensor(std).view(3,1,1)
        self.isTrain = isTrain
        self.transform = transform

        self.images = None
        if cache_images:
            self._cache_all_images(cache_workers)


    def __len__(self): return len(self.df)

    def _load_img(self, idx_path):
      i, path = idx_path

      if path is None or path == "":
        return i, None

      try:
        with Image.open(path) as img:
          img = img.convert("RGB")
          img = img.resize(image_size, Image.BILINEAR)
          if self.transform is not None:
            img = self.transform(img)
          # if self.isTrain:
          #   img = T.Compose([
          #       T.RandomResizedCrop(image_size, scale=(0.6, 1.0)),
          #       T.RandomHorizontalFlip(),
          #       T.RandomVerticalFlip(),
          #       T.ColorJitter(
          #         brightness=0.2,
          #         contrast=0.2,
          #         saturation=0.2,
          #         hue=0.02,
          #       ),
          #       T.RandomRotation(10),
          #   ])(img)

          # img = T.Compose([
          #   T.ToTensor(),
          #   T.Normalize(
          #       mean=self.mean,
          #       std=self.std,
          #   ),
          # ])(img)

        return i, img

      except Exception as e:
        print(e)
        return i, None

    def _cache_all_images(self, cache_workers: int):
        paths = self.df["image_path_fixed"].tolist()
        self.images = [None] * len(paths)
        bad_indicies = [None] * len(paths)

        print(f"Caching {len(paths)} images into RAM using {cache_workers} threads...\n")

        with ThreadPoolExecutor(max_workers=cache_workers) as ex:
            for i, img in tqdm(
                ex.map(self._load_img, [(i, p) for i, p in enumerate(paths)]),
                total=len(paths),
                desc="Caching dataset"
            ):
              if img is None:
                bad_indicies[i] = -1
                continue
              self.images[i] = img

        bad = [i for i, num in enumerate(bad_indicies) if num == -1]

        if len(bad) > 0:
          print(f"Dropping {len(bad)} samples with missing/unreadable images")
          self.df = self.df.drop(index=bad).reset_index(drop=True)
          # Keep only good tensors aligned with new df

          self.images = [img for img in self.images if img is not None]

        print("Cache complete — all images ready from RAM")

    def __getitem__(self, idx: int):
        label = int(self.df.iloc[idx]["label_id"])
        return self.images[idx], label


train_dataset = ReptiliaDataset(
  train_df, cache_images=True, cache_workers=num_workers*2, isTrain=True, transform=preprocess_train
)
val_dataset   = ReptiliaDataset(
  val_df, cache_images=True, cache_workers=num_workers, transform=preprocess_val
)
test_dataset  = ReptiliaDataset(test_df, cache_images=True, cache_workers=num_workers, transform=preprocess_val)

Caching 13879 images into RAM using 44 threads...



Caching dataset:   0%|          | 0/13879 [00:00<?, ?it/s]

Cache complete — all images ready from RAM
Caching 1982 images into RAM using 22 threads...



Caching dataset:   0%|          | 0/1982 [00:00<?, ?it/s]

Cache complete — all images ready from RAM
Caching 1753 images into RAM using 22 threads...



Caching dataset:   0%|          | 0/1753 [00:00<?, ?it/s]

Cache complete — all images ready from RAM


In [ ]:
def create_warmup_scheduler(optimizer, warmup_steps: int):
    """
    Linear warmup from 0 -> 1 over warmup_steps, then stays at 1.
    """
    def lr_lambda(step: int):
        if step < warmup_steps:
            return float(step) / float(max(1, warmup_steps))
        return 1.0

    return LambdaLR(optimizer, lr_lambda)


def train_one_epoch(
    model,
    loader,
    optimizer,
    scheduler,
    epoch,
    total_steps_done,
    use_bf16 = True,
):
    model.train()

    device        = fabric.device
    device_type   = device.type
    is_cuda       = (device_type == "cuda")
    is_xla        = (device_type == "xla")

    bf16_ok       = use_bf16 and (is_xla or (is_cuda and torch.cuda.is_bf16_supported()))
    use_scaler    = not bf16_ok
    dtype         = torch.bfloat16 if bf16_ok else torch.float16

    scaler        = amp.GradScaler(device=device_type, enabled=use_scaler)

    # Cache lookups for micro-efficiency (yes, it matters)
    _loss_fn      = F.cross_entropy
    _zero_grad    = optimizer.zero_grad
    _optim_step   = optimizer.step

    total_loss     = 0.0
    total_correct  = 0
    total_samples  = 0

    for images, labels in tqdm(loader, desc=f"Train Epoch {epoch}"):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        _zero_grad(set_to_none=True)
        with torch.autocast(device_type=device_type, dtype=dtype, enabled=scaler.is_enabled()):
            outputs = model(images)
            loss = _loss_fn(outputs, labels, label_smoothing=0.2)
        if use_scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            fabric.backward(loss)
            _optim_step()

        if scheduler is not None:
            scheduler.step()

        bs = labels.size(0)
        total_loss    += loss.item() * bs
        total_correct += (outputs.argmax(1) == labels).sum().item()
        total_samples += bs
        total_steps_done += 1

    avg_loss = total_loss / max(1, total_samples)
    train_acc = total_correct / max(1, total_samples)
    return total_steps_done, avg_loss, train_acc


@torch.no_grad()
def evaluate(model, loader):
    model.eval()

    total_loss = 0.0
    total_correct = 0
    total_examples = 0

    for images, labels in tqdm(loader, desc="Eval"):
        images = images.to(fabric.device.type, non_blocking=True)
        labels = labels.to(fabric.device.type, non_blocking=True)

        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)

        preds = outputs.argmax(dim=1)

        # accumulate for simple accuracy
        total_loss += loss.item() * labels.size(0)
        total_correct += (preds == labels).sum().item()
        total_examples += labels.size(0)

    if total_examples == 0:
        return 0.0, 0.0

    acc = total_correct / total_examples
    avg_loss = total_loss / total_examples

    return acc, avg_loss


In [ ]:
from google.colab import output
output.no_vertical_scroll()

EPOCHS = 40
WARMUP_STEPS = 5000

# weights = ViT_B_32_Weights.IMAGENET1K_V1

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=os.cpu_count()-3,
    persistent_workers=True,
    prefetch_factor=4,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=True,
    prefetch_factor=4,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=True,
    prefetch_factor=4,
)

# with fabric.init_module():
#     model_vit = vit_b_32(weights=weights)

# in_features = model_vit.heads.head.in_features
# model_vit.heads.head = nn.Sequential(
#     nn.Dropout(p=0.5),
#     nn.Linear(in_features, num_classes, device=fabric.device.type),
# )

LR = 0.0005
WEIGHT_DECAY = 0.0003
# optimizer_vit = AdamW(
#     model_vit.parameters(),
#     lr=LR,
#     weight_decay=WEIGHT_DECAY,
# )

optimizer_bio = AdamW(
    model_bio.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY,
)

# model, optimizer = fabric.setup(model_vit, optimizer_vit)
model, optimizer = fabric.setup(model_bio, optimizer_bio)




train_loader, val_loader, test_loader = fabric.setup_dataloaders(
    train_loader, val_loader, test_loader
)

# Steps per epoch = batches in train loader
steps_per_epoch = len(train_loader)
total_training_steps = EPOCHS * steps_per_epoch
print("Steps per epoch:", steps_per_epoch)
print("Total training steps:", total_training_steps)

scheduler_vit = create_warmup_scheduler(
    optimizer,
    warmup_steps=WARMUP_STEPS,
)

plateau_scheduler = ReduceLROnPlateau(
    optimizer,
    mode="max",       # because we want to maximize validation accuracy
    factor=0.001,
    threshold=0.01,
    patience=2,
    min_lr=1e-7,
)

history_vit = {
  "epoch": [],
  "val_acc": [],
  "val_loss": [],
  "train_acc": [],
  "train_loss": [],
  "lr": [],
  "steps": [],
  "total_steps": [],
  "train_durration": [],
  "val_durration": [],
  "overall_durration": [],
}

global_steps = 0
total_length = 0

# no_improv_count, last_val_acc = 0, 0.0
for epoch in range(1, EPOCHS + 1):
    use_warmup_this_epoch = global_steps < WARMUP_STEPS
    this_epoch_scheduler = scheduler_vit if use_warmup_this_epoch else None

    start = time.time()
    curr_total_steps, loss, acc = train_one_epoch(
        model,
        train_loader,
        optimizer,
        this_epoch_scheduler,
        epoch,
        total_steps_done=global_steps,
        use_bf16=True,
    )
    end_train = time.time()
    val_acc, val_loss = evaluate(model, val_loader)
    # val_acc, val_loss = evaluate(model, val_loader)
    end_val = time.time()

    total_length += end_val-start

    history_vit["epoch"].append(epoch)
    history_vit["val_acc"].append(val_acc)
    history_vit["val_loss"].append(val_loss)
    history_vit["train_acc"].append(acc)
    history_vit["train_loss"].append(loss)
    history_vit["lr"].append(optimizer.param_groups[0]["lr"])
    history_vit["steps"].append(curr_total_steps-global_steps)
    history_vit["total_steps"].append(curr_total_steps)
    history_vit["train_durration"].append(end_train-start)
    history_vit["val_durration"].append(end_val-end_train)
    history_vit["overall_durration"].append(total_length)

    global_steps = curr_total_steps

    if global_steps >= WARMUP_STEPS:
        plateau_scheduler.step(val_acc)

    print(f"Epoch {epoch} Steps {global_steps}/{total_training_steps} in {round(end_val-start, 2)}s - "
    f"Train/Val Accuracy: {acc*100:.2f}%|{val_acc*100:.2f}% LR: {optimizer.param_groups[0]["lr"]}")


<IPython.core.display.Javascript object>

Steps per epoch: 217
Total training steps: 8680


Train Epoch 1:   0%|          | 0/217 [00:00<?, ?it/s]

Eval:   0%|          | 0/31 [00:00<?, ?it/s]

Eval:   0%|          | 0/31 [00:00<?, ?it/s]

...

Epoch 39 Steps 8463/8680 in 11.17s - Train/Val Accuracy: 74.34%|54.34% LR: 1e-07


Train Epoch 40:   0%|          | 0/217 [00:00<?, ?it/s]

Eval:   0%|          | 0/31 [00:00<?, ?it/s]

Eval:   0%|          | 0/31 [00:00<?, ?it/s]

Epoch 40 Steps 8680/8680 in 11.19s - Train/Val Accuracy: 74.34%|54.34% LR: 1e-07
